# Azure AI Gateway Complete Deployment

This notebook demonstrates how to deploy all Azure AI Gateway resources in one command.

## What Gets Deployed

- **Step 1**: Core Infrastructure (APIM, App Insights, Log Analytics) ~15 min
- **Step 2**: AI Foundry Hubs + Models (3 regions) ~30 min
- **Step 3**: Supporting Services (Redis, Cosmos, Search) ~10 min
- **Step 4**: MCP Servers (Container Apps) ~5 min

**Total Time**: ~60 minutes

## Prerequisites

```bash
# Install required packages
pip install azure-identity azure-mgmt-resource azure-mgmt-cognitiveservices python-dotenv

# Login to Azure
az login
az account set --subscription "your-subscription-id"
```

## Step 1: Import and Configure

In [ ]:
import os
import sys
from pathlib import Path

# Add util directory to path if running from notebook
util_path = Path.cwd().parent / 'util'
if util_path.exists() and str(util_path) not in sys.path:
    sys.path.insert(0, str(util_path.parent))

from util import deploy_complete_infrastructure, DeploymentConfig, DeploymentProgress

print("✅ Imports successful")

## Step 2: Create Configuration

In [ ]:
# Load environment variables if .env file exists
env_file = Path('.env')
if env_file.exists():
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Loaded .env file")

# Get subscription ID
subscription_id = os.getenv('AZURE_SUBSCRIPTION_ID')
if not subscription_id:
    subscription_id = input("Enter your Azure Subscription ID: ").strip()

# Create deployment configuration
config = DeploymentConfig(
    subscription_id=subscription_id,
    resource_group='lab-master-lab',
    location='uksouth',
    # Optional: Customize these
    deploy_content_safety=True,
    deploy_mcp_servers=True
)

print("\n" + "="*70)
print("DEPLOYMENT CONFIGURATION")
print("="*70)
print(f"Subscription ID: {config.subscription_id}")
print(f"Resource Group: {config.resource_group}")
print(f"Location: {config.location}")
print(f"Resource Suffix: {config.resource_suffix}")
print(f"Deploy Content Safety: {config.deploy_content_safety}")
print(f"Deploy MCP Servers: {config.deploy_mcp_servers}")
print(f"Primary Models: {len(config.primary_models)}")
print(f"Secondary Models: {len(config.secondary_models)}")
print("="*70)

## Step 3: Define Progress Callback

In [ ]:
import time
from IPython.display import display, HTML, clear_output

class ProgressTracker:
    def __init__(self):
        self.steps = []
        self.start_time = time.time()

    def update(self, progress: DeploymentProgress):
        self.steps.append(progress)

        # Clear previous output
        clear_output(wait=True)

        # Status icons
        status_icons = {
            'pending': '⏳',
            'in_progress': '🔄',
            'completed': '✅',
            'failed': '❌'
        }

        # Build HTML table
        html = '<h3>Deployment Progress</h3>'
        html += '<table style="width:100%; border-collapse: collapse;">'
        html += '<tr style="background-color: #f0f0f0;">'
        html += '<th style="padding: 8px; text-align: left;">Step</th>'
        html += '<th style="padding: 8px; text-align: left;">Status</th>'
        html += '<th style="padding: 8px; text-align: left;">Message</th>'
        html += '<th style="padding: 8px; text-align: left;">Time</th>'
        html += '</tr>'

        for step in self.steps:
            icon = status_icons.get(step.status, '•')
            mins = int(step.elapsed_seconds / 60)
            secs = int(step.elapsed_seconds % 60)
            time_str = f"{mins}m {secs}s" if step.elapsed_seconds > 0 else "-"

            html += '<tr style="border-bottom: 1px solid #ddd;">'
            html += f'<td style="padding: 8px;">{step.step}</td>'
            html += f'<td style="padding: 8px;">{icon} {step.status}</td>'
            html += f'<td style="padding: 8px;">{step.message}</td>'
            html += f'<td style="padding: 8px;">{time_str}</td>'
            html += '</tr>'

            if step.error:
                html += '<tr>'
                html += f'<td colspan="4" style="padding: 8px; color: red;">❌ Error: {step.error}</td>'
                html += '</tr>'

        html += '</table>'

        # Overall elapsed time
        elapsed = time.time() - self.start_time
        mins = int(elapsed / 60)
        secs = int(elapsed % 60)
        html += f'<p><strong>Total elapsed time:</strong> {mins}m {secs}s</p>'

        display(HTML(html))

# Create tracker
tracker = ProgressTracker()
print("✅ Progress tracker initialized")

## Step 4: Deploy Infrastructure

**⚠️ This will take approximately 60 minutes**

The deployment will:
1. Verify prerequisites (Azure CLI, Bicep, authentication)
2. Create or verify resource group
3. Deploy core infrastructure (APIM, monitoring)
4. Deploy AI Foundry hubs and models
5. Deploy supporting services (Redis, Cosmos, Search)
6. Deploy MCP servers (Container Apps)

You can safely interrupt and re-run - already deployed resources will be skipped.

In [ ]:
# Deploy all infrastructure
try:
    outputs = deploy_complete_infrastructure(
        config,
        progress_callback=tracker.update
    )
    print("\n✅ DEPLOYMENT COMPLETE!")
except Exception as e:
    print(f"\n❌ DEPLOYMENT FAILED: {e}")
    print("Check deployment.log for details")
    raise

## Step 5: Save Outputs

In [ ]:
# Save environment file
outputs.to_env_file('master-lab.env')
print("✅ Environment file saved to: master-lab.env")

# Save JSON outputs
outputs.to_json('deployment-outputs.json')
print("✅ JSON outputs saved to: deployment-outputs.json")

## Step 6: Display Summary

In [ ]:
from IPython.display import display, Markdown

summary = f"""
## Deployment Summary

**Duration**: {outputs.deployment_duration_seconds/60:.1f} minutes  
**Timestamp**: {outputs.deployment_timestamp}  
**Resource Suffix**: {outputs.resource_suffix}

### Core Infrastructure

- **APIM Gateway URL**: `{outputs.apim_gateway_url}`
- **APIM Service Name**: `{outputs.apim_service_name}`
- **APIM Subscription Key**: `{outputs.apim_subscription_key[:20]}...`

### AI Foundry Hubs

- **Foundry 1 (uksouth)**: `{outputs.foundry1_endpoint}`
- **Foundry 2 (swedencentral)**: `{outputs.foundry2_endpoint}`
- **Foundry 3 (westeurope)**: `{outputs.foundry3_endpoint}`

### Supporting Services

- **Redis Cache**: `{outputs.redis_host}:10000`
- **Cosmos DB**: `{outputs.cosmos_endpoint}`
- **AI Search**: `{outputs.search_endpoint}`
- **Content Safety**: `{outputs.content_safety_endpoint or 'Not deployed'}`

### Container Infrastructure

- **Container Registry**: `{outputs.container_registry_login_server}`
- **MCP Servers**: {len(outputs.mcp_server_urls) if outputs.mcp_server_urls else 0} deployed

### Monitoring

- **Log Analytics Workspace**: `{outputs.log_analytics_workspace_id}`
- **Application Insights**: Configured

### Output Files

- ✅ `master-lab.env` - Environment variables
- ✅ `deployment-outputs.json` - Complete outputs
- ✅ `deployment.log` - Detailed log
"""

display(Markdown(summary))

## Step 7: Test APIM Gateway (Optional)

In [ ]:
import requests
import json

# Test APIM gateway with a simple request
def test_apim_gateway():
    if not outputs.apim_gateway_url:
        print("❌ APIM Gateway URL not available")
        return

    # Construct API endpoint (adjust based on your API configuration)
    url = f"{outputs.apim_gateway_url}/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview"

    headers = {
        'Ocp-Apim-Subscription-Key': outputs.apim_subscription_key,
        'Content-Type': 'application/json'
    }

    payload = {
        'messages': [
            {'role': 'user', 'content': 'Hello! This is a test from the deployment notebook.'}
        ],
        'max_tokens': 50
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()

        result = response.json()
        print("✅ APIM Gateway is working!")
        print(f"\nResponse: {result['choices'][0]['message']['content']}")

    except Exception as e:
        print(f"❌ Gateway test failed: {e}")
        print("This is expected if the API hasn't been configured yet.")

# Uncomment to test
# test_apim_gateway()

## Next Steps

1. Load the environment file: `source master-lab.env` (Linux/Mac) or review the file for Windows
2. Configure APIM APIs and policies
3. Test individual labs
4. Set up monitoring dashboards in Azure Portal

## Clean Up

To delete all deployed resources:

```bash
az group delete --name lab-master-lab --yes --no-wait
```